In [20]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [111]:
class ShortButteryfly:
    def __init__(self, chain):
        self.chain = chain
        self.call, self.put = self.get_atm_options()

        self.underlying_price = self.call["underprice"]
        # Because of exception handling, put and call strikes are always the same
        self.strike = self.call["strike"]
        self.premium = self.calc_premium()

    def get_atm_options(self):
        calls = self.chain[self.chain["right"] == "C"]
        puts = self.chain[self.chain["right"] == "P"]

        atm_call = calls.loc[np.abs(calls["delta"] - 0.50).idxmin()]
        atm_put = puts.loc[np.abs(puts["delta"] + 0.50).idxmin()]

        # Make sure selected options have the same strike. This should always be the case.
        if atm_call["strike"] != atm_put["strike"]:
            raise ValueError("Strike of computed ATM call and put is not identical")

        return atm_call, atm_put

    def calc_premium(self):
        call_mid = (self.call["bid"] + self.call["ask"]) / 2
        put_mid = (self.put["bid"] + self.put["ask"]) / 2
        return call_mid + put_mid

    def calc_pnl(self, underlying_price):
        if underlying_price > self.strike:
            pnl = self.premium - (underlying_price - self.strike)
        elif underlying_price < self.strike:
            pnl = self.premium - (self.strike - underlying_price)
        return pnl

def log_wealth_optim(f, pnl):
    """
    Returns the negative of log wealth for optimization
    """
    return -np.mean(np.log(1 + f * pnl))


def calc_kelly(position: ShortButteryfly, returns):
    prices = position.underlying_price * (1 + returns)
    pnl_func = np.vectorize(position.calc_pnl)
    pnls = pnl_func(prices)
    # Kelly function uses log of wealth, so it cannot be negative
    # This scales the pnl values so the lowest they ever go is 0
    # Because it's a monotonic transformation, the optimization
    # still finds the best point
    scaled_pnls = pnls / np.max(np.abs(pnls) + 1)

    initial = 0.50
    result = minimize(log_wealth_optim, x0=initial, args=(scaled_pnls), constraints=({"type": "ineq", "fun": lambda x: x},
                                                                                     {"type": "ineq", "fun": lambda x: 1 - x}))
    return result


In [112]:
chain = pd.read_csv("option_chain.csv")
position = ShortButteryfly(chain)

In [117]:
returns = np.linspace(-0.013, 0.013, 10000)

In [118]:
calc_kelly(position, returns)

     fun: -0.04088394248416916
     jac: array([0.00053435])
 message: 'Optimization terminated successfully'
    nfev: 7
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0.59379675])